# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [43]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier 
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [44]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [45]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [46]:
data = pd.read_csv("data/kddcup.data_10_percent", header=None, names = col_names)

In [47]:
data.shape

(494021, 42)

# 前処理
## カテゴリ化

In [48]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [49]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [50]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [51]:
data['label3'] = data.label.copy()

In [52]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop|mailbomb|apache2|processtable|udpstorm'),'label3'] = 'DoS'

In [53]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit|ps|xterm|sqlattack'),'label3'] = 'U2R'

In [54]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster|snmpgetattack|snmpguess|httptunnel|sendmail|named|xlock|xsnoop|worm'),'label3'] = 'R2L'

In [55]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan|mscan|saint'),'label3'] = 'Probe'

In [56]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

In [57]:
#joblib.dump(data,'dump/20171118/corrected.pkl')

## サンプリング

In [58]:
data = resample(data,n_samples=2000,random_state=0)

In [59]:
data.shape

(2000, 44)

## 数値化

In [60]:
le_protocol_type = preprocessing.LabelEncoder()

In [61]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [62]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [63]:
le_service = preprocessing.LabelEncoder()

In [64]:
le_service.fit(data.service)

LabelEncoder()

In [65]:
data.service = le_service.transform(data.service)

In [66]:
le_flag = preprocessing.LabelEncoder()

In [67]:
le_flag.fit(data.flag)

LabelEncoder()

In [68]:
data.flag = le_flag.transform(data.flag)

In [69]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,2000.000000,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2000.0,2000.00000,2000.0,2000.000000,...,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,62.213500,0.488500,12.034000,3.599000,3.435584e+03,606.287000,0.0,0.00750,0.0,0.012000,...,231.9170,189.213500,0.755880,0.032405,0.594735,0.007305,0.175355,0.174320,0.058020,0.056780
std,712.957848,0.591644,5.784857,0.953758,1.148363e+05,5315.351494,0.0,0.14985,0.0,0.154493,...,65.6843,105.264259,0.409118,0.115545,0.482721,0.043859,0.379430,0.379119,0.230094,0.228411
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.00000,0.0,0.000000,...,1.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,8.000000,4.000000,4.400000e+01,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,66.000000,0.460000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,8.000000,4.000000,5.200000e+02,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,19.000000,4.000000,1.032000e+03,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,14744.000000,2.000000,32.000000,5.000000,5.133876e+06,202976.000000,0.0,3.00000,0.0,3.000000,...,255.0000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [70]:
data.shape

(2000, 44)

## ラベルの分離

In [71]:
y_test_1 = data.label.copy() 

In [72]:
y_test_2 = data.label2.copy()

In [73]:
y_test_3 = data.label3.copy()

In [74]:
x_test= data.drop(['label','label2','label3'],axis=1)

In [75]:
x_test.shape

(2000, 41)

In [76]:
y_test_1.shape

(2000,)

In [77]:
y_test_2.shape

(2000,)

In [78]:
y_test_3.shape

(2000,)

## 標準化

In [79]:
ss = preprocessing.StandardScaler()

In [80]:
ss.fit(x_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [81]:
x_test = ss.transform(x_test)

In [82]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [83]:
pd.DataFrame(x_test,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2000.0,2.000000e+03,2000.0,2.000000e+03,...,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03
mean,1.274536e-16,1.367795e-16,-4.882761e-16,-2.858824e-16,3.262668e-17,3.601286e-18,0.0,2.375808e-16,0.0,-2.817746e-16,...,-1.356137e-16,-1.768585e-16,-9.126033e-17,1.248168e-16,3.029799e-16,-4.483913e-17,-1.521006e-16,1.263434e-16,1.183498e-16,2.871592e-16
std,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,0.0,1.000250e+00,0.0,1.000250e+00,...,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00
min,-8.728294e-02,-8.258718e-01,-2.080779e+00,-3.774439e+00,-2.992470e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,-3.516438e+00,-1.788457e+00,-1.848048e+00,-2.805248e-01,-1.232354e+00,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
25%,-8.728294e-02,-8.258718e-01,-6.975122e-01,4.205473e-01,-2.954145e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,-1.170809e+00,-7.233959e-01,-2.805248e-01,-1.232354e+00,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
50%,-8.728294e-02,-8.258718e-01,-6.975122e-01,4.205473e-01,-2.539539e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,6.251215e-01,5.968481e-01,-2.805248e-01,8.397520e-01,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
75%,-8.728294e-02,8.647562e-01,1.204479e+00,4.205473e-01,-2.093575e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,6.251215e-01,5.968481e-01,6.574867e-02,8.397520e-01,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
max,2.059793e+01,2.555384e+00,3.452288e+00,1.469294e+00,4.468728e+01,3.808221e+01,0.0,1.997498e+01,0.0,1.934558e+01,...,3.515113e-01,6.251215e-01,5.968481e-01,8.376311e+00,8.397520e-01,2.263922e+01,2.173922e+00,2.178436e+00,4.094924e+00,4.130525e+00


## 学習

In [84]:
#ニューラルネットワークの隠れ層の候補をいろいろ定義
parameters = {'hidden_layer_sizes' : [(100,), (100, 10), (100, 100, 10), (100, 100, 100, 10)]}

#ニューラルネットワークのベストな隠れ層を探索
clf = GridSearchCV(MLPClassifier(), parameters)
clf.fit(x_test, y_test_3)
clf.best_params_


{'hidden_layer_sizes': (100, 100, 100, 10)}

In [86]:
clf.best_estimator_

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [87]:
clf = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [88]:
clf.fit(x_test, y_test_3)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [89]:
pred = clf.predict(x_test)

In [90]:
print(classification_report(y_test_3, pred))
print(confusion_matrix(y_test_3, pred))

             precision    recall  f1-score   support

        DoS       1.00      1.00      1.00      1555
      Probe       1.00      1.00      1.00        19
        R2L       0.80      1.00      0.89         4
    normal.       1.00      1.00      1.00       422

avg / total       1.00      1.00      1.00      2000

[[1555    0    0    0]
 [   0   19    0    0]
 [   0    0    4    0]
 [   0    0    1  421]]
